In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


file = "/home/diya/personal/work/projects/bank fraud detection/bank_transactions_data.csv"
data = pd.read_csv(file)

#examining the first few rows of the csv file
print(data.head())

  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

In [ ]:
#checking for missing values in the dataset
print("missing values:\n" , data.isnull().sum()) #counts the true values for the .isnull() function ine each column

#store the columns which contain only numbers in a seperate variavle
num_columns = data.select_dtypes(include=['float64', 'int64']).columns

missing values:
 TransactionID              0
AccountID                  0
TransactionAmount          0
TransactionDate            0
TransactionType            0
Location                   0
DeviceID                   0
IP Address                 0
MerchantID                 0
Channel                    0
CustomerAge                0
CustomerOccupation         0
TransactionDuration        0
LoginAttempts              0
AccountBalance             0
PreviousTransactionDate    0
dtype: int64


In [ ]:
#from the "missing values:" we see that no column has any missing value and thus the dataset does not need any sort of "filling"
#converting "date" columns into date time format if in string format
date_columns = [col for col in data.columns if 'date' in col.lower()]
if date_columns:
    for col in date_columns:
        data[col] = pd.to_datetime(data[col])